In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt

sc.settings.verbosity = 3 
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['font.sans-serif'] = 'Arial'
sns.set_style('white', {'axes.grid' : False})

In [2]:
metadata = pd.read_csv('../GSE196830.meta')
batch_pool_dict = dict(zip(metadata['batch'],metadata['pool']))

In [3]:
herv_file_list = glob('/storage/zhufan/data/GSE196830/matrix/herv_gene_repeatmasker/*')
herv_adata_list = []
for herv_file in herv_file_list:
    gsm =  herv_file.split('/')[-1]
    pool = batch_pool_dict[gsm]
    herv = sc.read_10x_mtx(herv_file)
    herv.obs_names = herv.obs.reset_index()['index'].apply(lambda x : x.split('-')[0] + '-' + str(batch_pool_dict[gsm])).values
    herv.obs['batch'] = gsm
    herv.obs['pool'] = str(batch_pool_dict[gsm])
    herv_adata_list.append(herv)

--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading fr

In [4]:
herv = sc.concat(herv_adata_list)

In [5]:
sc.pp.filter_cells(herv, min_genes = 1)
sc.pp.filter_genes(herv, min_cells = 1)

filtered out 419471 genes that are detected in less than 1 cells


In [22]:
herv

AnnData object with n_obs × n_vars = 1543972 × 273288
    obs: 'batch', 'pool', 'n_genes'
    var: 'n_cells'

In [3]:
rna = sc.read_h5ad('./GSE196830_re_anno.h5ad')

In [23]:
common_index = rna.obs_names.intersection(herv.obs_names)
rna = rna[common_index].copy()
herv = herv[common_index].copy()

In [24]:
herv

AnnData object with n_obs × n_vars = 1174452 × 273288
    obs: 'batch', 'pool', 'n_genes'
    var: 'n_cells'

In [37]:
adata = herv.copy()

In [38]:
sc.pp.filter_genes(adata, min_cells = 20)

filtered out 216620 genes that are detected in less than 20 cells


In [43]:
out = '/storage/liuyi/06.herv/herv_adata/herv_adata_product'
adata.obs[['indiv_ID','sex','age','celltype','age_stage','age_phase']] = rna.obs[['indiv_ID','sex','age','celltype','age_stage','age_phase']]
herv_info = pd.read_csv('/storage/liuyi/06.herv/herv_adata/herv_adata_product/herv_info/HERV_gene_info.txt',sep = '\t',header=None,index_col=4,names=['chrom','func','start','end'])
adata.var[['chrom','start','end']] = herv_info[['chrom','start','end']]
adata.var[['chrom','start','end']].to_csv(f'{out}/effect_herv.bed',sep = '\t',index = None,header=None)

In [47]:
out = '/storage/liuyi/06.herv/herv_adata/herv_adata_product/all_herv'
for age_phase in ['adult','aging','aged']:
    tmp_adata = adata[adata.obs['age_phase'] == age_phase].copy()
    sc.pp.filter_genes(tmp_adata, min_cells = 10)
    sc.pp.calculate_qc_metrics(tmp_adata,percent_top=None, log1p=False, inplace=True)
    tmp_df = tmp_adata.var[['chrom','start','end','total_counts']].copy()
    tmp_df['total_counts'] = tmp_df['total_counts']/tmp_df['total_counts'].sum()*1e5
    tmp_df.to_csv(f'{out}/effect_herv_{age_phase}.bed',sep = '\t',index = None,header=None)

filtered out 30449 genes that are detected in less than 10 cells
filtered out 2503 genes that are detected in less than 10 cells
filtered out 6586 genes that are detected in less than 10 cells


In [48]:
annotation = pd.read_csv('/storage/liuyi/06.herv/herv_adata/herv_adata_product/herv_info/effect_herv_annotated.txt',sep = '\t')
annotation.index = adata.var_names
adata.var[['width','annotation','distanceToTSS','gene_name']] = annotation[['width','annotation','distanceToTSS','SYMBOL']]

In [50]:
sc.write('herv.anno.h5ad',adata)

In [3]:
herv= sc.read_h5ad('herv.anno.h5ad')